In [ ]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import re
import numpy as np
import random

In [ ]:
df1 = pd.read_csv("/content/animelist.csv")
df2 = pd.read_csv("/content/anime.csv")

In [ ]:
#เลือก columns ที่จะใช้
anime = df2.iloc[:,[0, 1]]
#เปลี่ยนชื่อ columns ให้ตรงกับ df1
anime.columns = ['anime_id', 'Name']

In [ ]:
#innerjoin ระหว่าง df1 กับ anime
anime_complete = pd.merge(df1, anime, on='anime_id')
#check ตัว NaN (Not a Number)
anime_complete.isna().sum()

In [ ]:
anime_complete

In [ ]:
#สร้าง function ลบตัวอักษรพิเศษ
def text_cleaning(text):
    text = re.sub(r'&quot;', '', text)# "
    text = re.sub(r'.hack//', '', text)#.hack//
    text = re.sub(r'&#039;', '', text)# '
    text = re.sub(r'A&#039;s', '', text)#A's
    text = re.sub(r'I&#039;', 'I\'', text)#I'
    text = re.sub(r'&amp;', 'and', text)#&
    return text

In [ ]:
#สร้างตัว copy
anime_features =  anime_complete.copy()
#ลบตัวอักษรพิเศษใน column "anime_title"
anime_features['Name'] = anime_features['Name'].apply(text_cleaning)

In [ ]:
user_id_count = anime_features['user_id'].value_counts()
user_id_count = user_id_count[user_id_count >= 100]#กรอง user ที่รีวิวมากกว่า 100 ครั้ง
test_index_num = user_id_count.sample(frac = 0.3).index#แบ่งเป็น test 30%
anime_features = anime_features[anime_features['user_id'].isin(user_id_count.index)]

In [ ]:
test = anime_features[anime_features['user_id'].isin(test_index_num)]#ถ้าใน column user_id มีตัวใน test_index_num ให้ใส่เข้าไป
train = anime_features[anime_features['user_id'].isin(test_index_num) == False]#ถ้าใน column user_id ไม่ได้มีตัวใน test_index_num ให้ใส่เข้าไป

In [ ]:
test = test.sort_values("user_id")
test = test[test["rating"] > 8.0]# rating มากกว่า 8
test = test.loc[:,["user_id","Name"]]# ลด column เหลือ แค่ 2 column

In [ ]:
ls_test = list(test.groupby("user_id"))

In [ ]:
train_pivot = train.pivot_table(index='Name', columns = 'user_id', values = 'rating').fillna(0)

In [ ]:
#เปลี่ยนจาก series เป็น csr_matrix (Compressed Sparse Row Matrix) เพื่อให้เวลา fit ข้อมูลใน Knn มันเร็วขึ้น
anime_matrix = csr_matrix(train_pivot.values)

In [ ]:
#fit ข้อมูล
model_knn2 = NearestNeighbors(algorithm='brute',metric="cosine")#ใช้วิธีแบบ cosine similarity
model_knn2.fit(anime_matrix)

In [ ]:
def give_ls_knn(model=model_knn2,anime_title = [np.random.choice(train_pivot.index)],anime_pivot=train_pivot, num = 100):
    a = np.array([[]])
    b = np.array([[]])
    # Print the selected anime title
    print("*********************************************************************************")
    print(f"Selected anime title: {anime_title}")
    # Find the row index of the selected anime title
    # ใช้ knn โดยหาว่าตัวไหนใกล้มันที่สุด
    for var in anime_title:
        query_index = anime_pivot.index.get_loc(var)#เปลี่ยนชื่อ anime เป็น index 
        distances, indices = model.kneighbors(anime_pivot.iloc[query_index, :].values.reshape(1, -1), n_neighbors=num) #เลือกมาใกล้ตัวที่ใกล้เคียงกัน
        #เอาแต่ละรอบมารวมกันเป็น Array เดียว
        a = np.concatenate((a,distances),axis = 1)
        b = np.concatenate((b,indices), axis = 1)
  
    con = np.transpose(np.concatenate((b,a), axis = 0)) #Transpose
    df = pd.DataFrame(con, columns= ["Indices", "Distances"])#สร้าง Dataframe
  
    #หาว่า anime เรื่องไหนไม่อยู่ใกล้ภายใน 100 ตัวแรกของ anime ที่อยุ่ใน anime_title ทั้งหมด
    df2 = df.groupby("Indices").count() != len(anime_title)
    #เอา anime_id จากเงื่อนไขข้างบนแปลงเป็น list
    ls1 = [int(var) for var in list(df.groupby("Indices").sum()["Distances"][df2["Distances"]].index)]
    #แปลง index ของ df ให้เป็นตาม anime_id
    df.index = ([int(var) for var in list(df["Indices"])])
    #drop ตัวที่เข้าเงื่อนไขออก
    for var in ls1:
        df = df.drop(var)
  
    #sum distance ทั้งหมดของแต่ละตัวและเรียงตาม Distance จากน้อยไปมาก
    df3 = df.groupby("Indices").sum().apply(lambda x: x).sort_values(['Distances','Indices'])
    #แยก Indices เป็น list
    indices = [int(var) for var in list(df3.index)][len(anime_title):len(anime_title)+20]
    #แยก Distances เป็น list
    distances = list(df3["Distances"])[len(anime_title):len(anime_title)+20]

    minest = []
    # หาว่ามีเรื่องไหนจาก 20 เรื่อง ที่แนะนำพิเศษสำหรับ กลุ่มเมะ ใน anime_title อันไหนเป็นพิเศษ
    for var2 in range(len(indices)):
        check = a[np.where(b == indices[var2])]#Distance ของตัวนั้นจาก แต่ละเรื่องใน anime_title
        add = []
        scoring = distances[var2]*((1/len(anime_title))*0.75)#เกณฑ์
        if len(check[np.where(check < scoring)]) != 0:
            for var in check[np.where(check < scoring)]:
                add.append(anime_title[list(check).index(var)])#เช็คว่าตัวที่น้อยกว่าเกณฑ์มาจากเรื่องไหน แล้วเพิ่มเข้า add
            minest.append(add)#เพิ่ม add เข้าไปใน minest
        else:
            minest.append(len(anime_title))
    ans = []
    print("*********************************************************************************")
    #print ชื่อ anime ที่ recommend
    for i, (distance, index) in enumerate(zip(distances, indices)):#len(anime_title): คือการตัดตัวเรื่องจาก anime_title ออก
        txt = ""
        if minest[i] != len(anime_title) and len(anime_title)!= 1:#เช็คว่าเรื่องนั้น แนะนำพิเศษสำหรับอนิเมะเรื่องไหน หรือไม่
            txt = "Significant from :"
            if len(minest[i])!= 1:
                for var in minest[i]:
                    txt += f" {var}"
            else:
                txt += f" {minest[i][0]}"
        print(f"{i+1:02d}: {anime_pivot.index[index]}, with total distance of {distance:.2f}")
        ans.append(anime_pivot.index[index])
        if txt != "":
            print(txt)
        print("---------------------------------------------------------------------------------")
    return ans

In [ ]:
score = 0
for var in ls_test:
    ls1 = var[1]["Name"].values[5:]
    ls2 = give_ls_knn(anime_title = list(var[1]["Name"].values[:5]))
    for i in ls1:
        if i in ls2:
            score += 1
            break
    print(score)

In [ ]:
print(f"Accuracy {score/len(ls_test)}")

In [ ]:
give_ls_knn(anime_title=["Naruto","One Punch Man"], model=model_knn2)

In [ ]:
give_ls_knn(anime_title=["Tensei shitara Slime Datta Ken","Boruto: Naruto Next Generations"], model=model_knn2)

In [ ]:
give_ls_knn(anime_title=["Boruto: Naruto Next Generations"], model=model_knn2)

In [ ]:
print(list([1,2,2]))